### MID 25
### Taking ant1.3 as training data.
### ant1.4 as testing data.
### Min max scaling done to few columns: ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
### PCA used for feature reduction, n_components = 10.
### Oversampling and smote methods used to increase size of training data.
### GRU model used, model type is 4 as per the BTP documentation spreadsheet.
### np.rint() used on predictions

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import GRU, Dense
import tensorflow as tf
import csv
from sklearn.decomposition import PCA

In [3]:
# Loading data. 
train_data_path = "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.3.csv"
test_data_path = "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.4.csv"
train_data_name = "ant-1.3"
test_data_name = "ant-1.4"
ant_1_3 = pd.read_csv(train_data_path)
ant_1_4 = pd.read_csv(test_data_path)
files = ["/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.3.csv", "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.4.csv"]
combined_data = pd.concat(map(pd.read_csv, files))

In [4]:
# Applying Min Max Scaling.
scaler = MinMaxScaler()
MinMaxScaler(copy=True, feature_range=(0, 1))
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
combined_data[cols_to_norm] = MinMaxScaler().fit_transform(combined_data[cols_to_norm])

In [5]:
combined_data

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,ant,1.3,org.apache.tools.ant.taskdefs.ExecuteOn,0.142857,0.6,0.050,0.102941,0.214286,0.011856,0.014815,...,1.000000,0.111111,0.885057,0.232323,0.75,0.363636,0.165951,0.085714,0.209085,0
1,ant,1.3,org.apache.tools.ant.DefaultLogger,0.181818,0.0,0.025,0.058824,0.163265,0.020033,0.029630,...,1.000000,0.000000,0.000000,0.307692,0.00,0.000000,0.080979,0.171429,0.269903,2
2,ant,1.3,org.apache.tools.ant.taskdefs.TaskOutputStream,0.038961,0.2,0.000,0.007353,0.045918,0.000000,0.000000,...,1.000000,0.111111,0.714286,0.666667,0.25,0.090909,0.083267,0.028571,0.109529,0
3,ant,1.3,org.apache.tools.ant.taskdefs.Cvs,0.155844,0.4,0.000,0.088235,0.188776,0.013083,0.000000,...,1.000000,0.111111,0.770833,0.458333,0.00,0.000000,0.115693,0.085714,0.232742,0
4,ant,1.3,org.apache.tools.ant.taskdefs.Copyfile,0.077922,0.4,0.000,0.029412,0.107143,0.000409,0.000000,...,1.000000,0.000000,0.880952,0.416667,0.50,0.181818,0.100881,0.028571,0.136898,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,ant,1.4,org.apache.tools.ant.TaskAdapter,0.064935,0.4,0.000,0.051471,0.096939,0.001635,0.029630,...,0.000000,0.000000,0.902439,0.400000,0.25,0.090909,0.138351,0.028571,0.131428,0
174,ant,1.4,org.apache.tools.ant.taskdefs.rmic.DefaultRmic...,0.220779,0.0,0.075,0.110294,0.336735,0.036795,0.029630,...,1.000000,0.222222,0.000000,0.197917,0.00,0.000000,0.172373,0.400000,0.396221,0
175,ant,1.4,org.apache.tools.ant.IntrospectionHelper,0.298701,0.0,0.000,0.227941,0.418367,0.082175,0.162963,...,0.444444,0.000000,0.000000,0.318182,0.00,0.000000,0.219724,0.685714,0.378561,0
176,ant,1.4,org.apache.tools.ant.taskdefs.compilers.Defaul...,0.142857,0.0,0.200,0.161765,0.438776,0.006132,0.059259,...,1.000000,0.777778,0.000000,0.266667,0.00,0.000000,0.373390,0.628571,0.761689,1


In [6]:
# define transform
components = 10
pca = PCA(n_components=components)
# prepare transform on dataset
pca.fit(combined_data[cols_to_norm])
# apply transform to dataset
transformed = pca.transform(combined_data[cols_to_norm])

In [7]:
transformed = pd.DataFrame(transformed)
transformed 

,0,1,2,3,4,5,6,7,8,9
0,-0.818904,0.255541,0.173010,-0.085265,-0.298426,-0.041536,-0.029952,0.060052,-0.061844,0.129277
1,0.211850,-0.596714,-0.058649,-0.060459,-0.059547,-0.063873,-0.088019,0.048838,0.136331,-0.023472
2,-0.310718,-0.021222,-0.538803,-0.015225,-0.015308,0.025091,0.105172,-0.187430,-0.136162,-0.004927
3,-0.328366,0.034184,-0.105838,-0.033729,0.314725,-0.066018,-0.185431,0.118595,-0.020596,-0.020034
4,-0.516435,0.234332,-0.202375,-0.122822,-0.155654,-0.028101,-0.014558,-0.023515,0.081734,0.039900
...,...,...,...,...,...,...,...,...,...,...
298,0.126544,0.701796,0.109947,-0.104515,0.057198,-0.305038,0.278453,-0.261710,-0.178725,-0.005567
299,0.071521,-0.725060,0.254031,0.156662,0.066015,-0.159664,-0.159516,0.152804,-0.060837,0.065179
300,0.276368,-0.485600,0.763979,0.403732,0.309357,-0.081984,-0.039603,-0.037146,-0.153829,0.234048
301,-0.023641,-0.851754,0.470180,0.584599,0.163868,-0.184260,-0.144292,0.435575,-0.433720,0.035253


In [8]:
train_data_index_list = list(range(ant_1_3.shape[0]))
train_data_index_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124]

In [9]:
test_data_index_list = list(range(ant_1_3.shape[0], ant_1_3.shape[0] + ant_1_4.shape[0]))
test_data_index_list

[125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 284,
 285,
 286,
 287,
 288,
 289,
 290,
 291

In [10]:
X_train = transformed[transformed.index.isin(train_data_index_list)]
X_train = np.array(X_train)

X_test = transformed[transformed.index.isin(test_data_index_list)]
X_test = np.array(X_test)

In [11]:
Y_train = combined_data[transformed.index.isin(train_data_index_list)]
Y_train = Y_train['bug']

In [12]:
Y_test = combined_data[transformed.index.isin(test_data_index_list)]
Y_test = Y_test['bug']

In [13]:
# Applying over sampling and SMOTE to training data for augmentation.
ros = RandomOverSampler(random_state=0)
X_train, Y_train = ros.fit_resample(X_train, Y_train)

smt = SMOTE()
X_train, Y_train = smt.fit_resample(X_train, Y_train)

train_x = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
test_x = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [14]:
train_x

array([[[-0.81890418,  0.25554076,  0.17301012, ...,  0.06005162,
         -0.06184364,  0.12927686]],

       [[ 0.21184961, -0.59671433, -0.0586493 , ...,  0.04883811,
          0.13633057, -0.02347176]],

       [[-0.31071828, -0.02122237, -0.53880307, ..., -0.18743043,
         -0.13616219, -0.00492705]],

       ...,

       [[ 0.13335335, -0.65412279,  0.11619592, ...,  0.01685192,
          0.05180648,  0.15503896]],

       [[ 0.13335335, -0.65412279,  0.11619592, ...,  0.01685192,
          0.05180648,  0.15503896]],

       [[-0.64581609,  0.16459048, -0.08562776, ..., -0.01983011,
         -0.08398178,  0.10540274]]])

In [15]:
Y_train

0      0
1      2
2      0
3      0
4      0
      ..
415    3
416    3
417    3
418    3
419    3
Name: bug, Length: 420, dtype: int64

In [16]:
train_y = Y_train.to_numpy()
train_y

array([0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 3, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [17]:
test_x

array([[[-0.43555707,  0.00661348, -0.00150416, ...,  0.0810492 ,
         -0.04502729, -0.04694391]],

       [[-0.58595064,  0.25955952, -0.13917763, ..., -0.0635911 ,
          0.00664355,  0.21386638]],

       [[ 0.27632365, -0.50600654, -0.32239189, ..., -0.05089842,
          0.05516215, -0.0075081 ]],

       ...,

       [[ 0.27636816, -0.48560036,  0.76397859, ..., -0.03714568,
         -0.1538287 ,  0.23404773]],

       [[-0.02364086, -0.85175387,  0.47017971, ...,  0.43557536,
         -0.43372011,  0.03525348]],

       [[-0.30582691,  0.13031094, -0.51606078, ..., -0.0809641 ,
         -0.00836604, -0.0343912 ]]])

In [18]:
Y_test

0      0
1      0
2      0
3      0
4      0
      ..
173    0
174    0
175    0
176    1
177    0
Name: bug, Length: 178, dtype: int64

In [19]:
test_y = Y_test.to_numpy()
test_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 3, 0, 0, 0, 0, 3, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0])

In [20]:
# Designing and initializing the model.
model = Sequential()
model.add(GRU(100, input_shape = (1,components), dropout = 0.2, return_sequences=True))
model.add(GRU(80, dropout = 0.2, return_sequences=True))
model.add(GRU(60, dropout = 0.2, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = 'adam' , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 1, 100)            33600     
_________________________________________________________________
gru_1 (GRU)                  (None, 1, 80)             43680     
_________________________________________________________________
gru_2 (GRU)                  (None, 60)                25560     
_________________________________________________________________
dense (Dense)                (None, 1)                 61        
Total params: 102,901
Trainable params: 102,901
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Fitting the model on training data.
history = model.fit(train_x, train_y, epochs = 100, batch_size = 128)

Epoch 1/100
4/4 [==============================] - 0s 3ms/step - loss: 3.4458 - mse: 3.4458 - mae: 1.4902 - root_mean_squared_error: 1.8563 - mean_squared_logarithmic_error: 0.8743
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 3.2412 - mse: 3.2412 - mae: 1.4501 - root_mean_squared_error: 1.8003 - mean_squared_logarithmic_error: 0.7754
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 2.9711 - mse: 2.9711 - mae: 1.3958 - root_mean_squared_error: 1.7237 - mean_squared_logarithmic_error: 0.6605
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 2.6064 - mse: 2.6064 - mae: 1.3141 - root_mean_squared_error: 1.6144 - mean_squared_logarithmic_error: 0.5283
Epoch 5/100
4/4 [==============================] - 0s 3ms/step - loss: 2.1704 - mse: 2.1704 - mae: 1.2005 - root_mean_squared_error: 1.4732 - mean_squared_logarithmic_error: 0.4016
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 1.6706 - mse: 1.6706 - m

4/4 [==============================] - 0s 3ms/step - loss: 0.8263 - mse: 0.8263 - mae: 0.7574 - root_mean_squared_error: 0.9090 - mean_squared_logarithmic_error: 0.1815
Epoch 47/100
4/4 [==============================] - 0s 3ms/step - loss: 0.7798 - mse: 0.7798 - mae: 0.7394 - root_mean_squared_error: 0.8830 - mean_squared_logarithmic_error: 0.1800
Epoch 48/100
4/4 [==============================] - 0s 3ms/step - loss: 0.8484 - mse: 0.8484 - mae: 0.7645 - root_mean_squared_error: 0.9211 - mean_squared_logarithmic_error: 0.1980
Epoch 49/100
4/4 [==============================] - 0s 3ms/step - loss: 0.8507 - mse: 0.8507 - mae: 0.7637 - root_mean_squared_error: 0.9223 - mean_squared_logarithmic_error: 0.1952
Epoch 50/100
4/4 [==============================] - 0s 3ms/step - loss: 0.8355 - mse: 0.8355 - mae: 0.7602 - root_mean_squared_error: 0.9141 - mean_squared_logarithmic_error: 0.1881
Epoch 51/100
4/4 [==============================] - 0s 3ms/step - loss: 0.8142 - mse: 0.8142 - mae: 0.7

4/4 [==============================] - 0s 3ms/step - loss: 0.7475 - mse: 0.7475 - mae: 0.6850 - root_mean_squared_error: 0.8646 - mean_squared_logarithmic_error: 0.1746
Epoch 92/100
4/4 [==============================] - 0s 3ms/step - loss: 0.7687 - mse: 0.7687 - mae: 0.7004 - root_mean_squared_error: 0.8768 - mean_squared_logarithmic_error: 0.1801
Epoch 93/100
4/4 [==============================] - 0s 4ms/step - loss: 0.7569 - mse: 0.7569 - mae: 0.7003 - root_mean_squared_error: 0.8700 - mean_squared_logarithmic_error: 0.1764
Epoch 94/100
4/4 [==============================] - 0s 3ms/step - loss: 0.7214 - mse: 0.7214 - mae: 0.6845 - root_mean_squared_error: 0.8493 - mean_squared_logarithmic_error: 0.1643
Epoch 95/100
4/4 [==============================] - 0s 3ms/step - loss: 0.7498 - mse: 0.7498 - mae: 0.7094 - root_mean_squared_error: 0.8659 - mean_squared_logarithmic_error: 0.1635
Epoch 96/100
4/4 [==============================] - 0s 3ms/step - loss: 0.7561 - mse: 0.7561 - mae: 0.7

In [23]:
# Getting the predictions.
predictions_y = model.predict(test_x)
predictions_y

array([[1.2128844 ],
       [2.1001894 ],
       [0.5793765 ],
       [1.365251  ],
       [1.7998129 ],
       [1.272622  ],
       [1.4693505 ],
       [0.        ],
       [0.94844353],
       [1.5052599 ],
       [1.5790235 ],
       [2.2282186 ],
       [0.887205  ],
       [1.1893156 ],
       [1.9562536 ],
       [0.2763195 ],
       [1.0058887 ],
       [0.        ],
       [0.        ],
       [0.87801856],
       [0.99994624],
       [0.5997076 ],
       [0.6850769 ],
       [1.7849791 ],
       [2.000596  ],
       [1.791516  ],
       [1.2346964 ],
       [0.7122446 ],
       [0.7315466 ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [1.4537289 ],
       [0.43116868],
       [0.        ],
       [1.4030751 ],
       [0.        ],
       [0.81705755],
       [0.6251728 ],
       [1.5417311 ],
       [0.        ],
       [0.        ],
       [0.2870173 ],
       [0.        ],
       [0.        ],
       [0.90504277],
       [1.618

In [24]:
# Rounding off the predictions to nearest
#integer as count of bugs is an integer.
predictions_y_round = np.rint(predictions_y)
predictions_y_round 

array([[1.],
       [2.],
       [1.],
       [1.],
       [2.],
       [1.],
       [1.],
       [0.],
       [1.],
       [2.],
       [2.],
       [2.],
       [1.],
       [1.],
       [2.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [2.],
       [2.],
       [2.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [2.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [2.],
       [3.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [2.],
       [2.],
       [0.],
       [0.],
       [1.],
       [1.],
       [2.],
       [0.],
       [2.],
       [1.],
       [2.],
       [2.],
       [0.],
       [0.],
       [2.],
       [0.],
       [2.],
       [1.],
       [2.],
       [2.],
       [2.],

In [25]:
# Viewing the number of predictions.
predictions_y_round.shape

(178, 1)

In [26]:
# Getting the sum of all the predictions obtained to used while obtaining FPA
s = 0
for  t in range(predictions_y_round.shape[0]):
    s+=predictions_y_round[t]
s

array([167.], dtype=float32)

In [27]:
# Obtaining the value of FPA metric for the model
Fpa = 0
for  t in range(predictions_y_round.shape[0]):
        x = 0
        for j in range( predictions_y_round.shape[0]-t+1, predictions_y_round.shape[0]):
               x = x + predictions_y_round[j]
        
        x = (x/s)/predictions_y_round.shape[0]
        Fpa = Fpa + x
Fpa

array([0.49431464], dtype=float32)

In [28]:
# Obtaining the value of CLC metric for the model
previous_obtained = predictions_y_round[predictions_y_round.shape[0] - 1]/s

CLC = 0
for i in range(predictions_y_round.shape[0]):
    if(i==0):
        CLC += 0 + previous_obtained
    else:
        additional = (predictions_y_round[predictions_y_round.shape[0] - 1 - i])/s
        CLC += 2*previous_obtained + additional
        previous_obtained += additional
        
CLC/=(2*predictions_y_round.shape[0])
CLC

array([0.5027083], dtype=float32)

In [29]:
# Getting direct metric results using the metrics given to model.
score = model.evaluate(test_x, test_y)
print(history.history.keys())
model.test_on_batch(test_x, test_y)
model.metrics_names
#print("\n%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[2], score[2]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[3], score[3]*100))


6/6 [==============================] - 0s 1ms/step - loss: 1.1861 - mse: 1.1861 - mae: 0.8551 - root_mean_squared_error: 1.0891 - mean_squared_logarithmic_error: 0.4216
dict_keys(['loss', 'mse', 'mae', 'root_mean_squared_error', 'mean_squared_logarithmic_error'])


['loss',
 'mse',
 'mae',
 'root_mean_squared_error',
 'mean_squared_logarithmic_error']

In [30]:
# Saving the model
model_id = 25
path_to_save = '/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/saved_models/ant1.3_ant1.4_model5/ant1.3_ant1.4_model5_4'
model.save(path_to_save)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/saved_models/ant1.3_ant1.4_model5/ant1.3_ant1.4_model5_4/assets


In [31]:
# Writing the results to csv file.
heading = ['model_id', 'train_data_name', 'test_data_name'] + model.metrics_names + ['fpa', 'clc']
score = [model_id, train_data_name, test_data_name] + score + [float(Fpa) , float(CLC)]

In [32]:
heading

['model_id',
 'train_data_name',
 'test_data_name',
 'loss',
 'mse',
 'mae',
 'root_mean_squared_error',
 'mean_squared_logarithmic_error',
 'fpa',
 'clc']

In [33]:
score

[25,
 'ant-1.3',
 'ant-1.4',
 1.1860753297805786,
 1.1860753297805786,
 0.8550759553909302,
 1.0890707969665527,
 0.42155712842941284,
 0.49431464076042175,
 0.5027083158493042]

In [34]:
# Writing the results to csv file.
with open(path_to_save + '_metric_results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(heading)
    writer.writerow(score)


In [35]:
# Loading the model and to make sure that the model is saved properly.
model_loaded = load_model(path_to_save)